Project by Daniele Asciutti and Samuele Borgonovo

## Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes. 
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms. 
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh. 

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons. 

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached mip-1.14.1-py3-none-any.whl (15.3 MB)


SETS:
  
*   $F$ = {1,...,5} (i) Farms
*   $Q$ = {1,...,8} (j) Trimesters
*   $Q_0$ = {0,...,8} 

PARAMETERS:
  
*   $p_{ij}$ production of the farm i in the trimester j
*   $c_i$ cost per ton of production of farm i
*   $s_i$ storage cost per ton of production of farm i
*   $d_i$ percentage of dry matter of production of farm i
*   $e$ energy per ton of burned material
*   $es$ price of energy sold  \$/KWh
*   $max$ max energy production in a trimester  KWh
*   $ms$ max tons that can be stored

VARIABLES:

*   $x_{ij}$ quantity of product i bought in the trimester j 
*   $y_{ij}$ quantity of product i burnt in the trimester j
*   $z_{ij}$ quantity of product i stored in the trimester j

CONSTRAINTS:


*   $z_{i,0}=0 \space\space  \forall i \in F $ 
*   $y_{i,j} = x_{i,j} + z_{i,j-1} - z_{i,j}\space\space \forall i \in F,\space \forall j \in Q$ 
*   $x_{i,j} \le p_{i,j} \space\space \forall i \in F,\space \forall j \in Q$
*   $\sum_{i \in F} z_{i,j} \le ms \space\space \forall j \in Q$
*   $\sum_{i \in F} y_{i,j}*d_{i} \ge  20*\sum_{i \in F} y_{i,j} \space\space \forall j \in Q$
*   $\sum_{i \in F} y_{i,j}*d_{i} \le  40*\sum_{i \in F} y_{i,j} \space\space \forall j \in Q$
*   $\sum_{i \in F} y_{i,j}*e \le max \space\space    \forall j \in Q$

OBJECTIVE FUNCTION:


> $\sum_{i \in F} \sum_{j \in Q} y_{i,j}*es*e - z_{i,j}*s_{i} - x_{i,j}*c_{i}$



In [8]:
p =\
[[700,1500,700,0,0,700,1500,0],
   [1350,0,450,0,1350,0,450,0],
   [0,1500,1500,0,0,1500,1500,0],
   [820,1560,820,0,820,1560,820,0],
   [0,680,1080,0,0,680,1080,0]]
              #tons of production of farm i (row) in the trimester j (column)
column=8
rows=5
c = [0.20,0.18,0.19,0.21,0.23]   #cost per ton of production of farm i
s = [0.002,0.012,0.007,0.011,0.015]   #storage cost per ton of product of farm i
d = [15,28,35,37,42]            # percentage of dry matter of product of farm i
e = 421.6    #energy per ton of burned material  KWh 
es = 0.28   #price of energy sold   $/KWh
max = 1950000  #max energy production on a trimester  KWh
ms = 500    #max tons that can be stored


import mip

m = mip.Model()
x = {(i,j): m.add_var() for i in range(1,rows+1) for j in range(1,column+1)} #quantity of prod i bought in trimester j
z = {(i,j): m.add_var() for i in range(1,rows+1) for j in range(column+1)} #quantity of prod i stored in trimester j
y = {(i,j): m.add_var() for i in range(1,rows+1) for j in range(1,column+1)} #quantity of prod i burnt in trimester j

#Constraints
for i in range(1,rows+1):
  m.add_constr(z[i,0] <= 0)     #storage in trimester 0 must be zero

for i in range(1,rows+1):
  for j in range(1,column+1):
    m.add_constr(y[i,j] - x[i,j] - z[i,j-1] + z[i,j] <= 0)     #multi-period constraint

for i in range(1,rows+1):
  for j in range(1,column+1):
    m.add_constr(x[i,j] <= p[i-1][j-1])    #max buy

for j in range (1,column+1):
  m.add_constr(mip.xsum(z[i,j] for i in range(1,rows+1)) <= ms)  #max storage
  m.add_constr(mip.xsum(y[i,j]*d[i-1] for i in range(1,rows+1)) >= 20*mip.xsum(y[i,j] for i in range(1,rows+1)))  #dry matter percentage >=20
  m.add_constr(mip.xsum(y[i,j]*d[i-1] for i in range(1,rows+1)) <= 40*mip.xsum(y[i,j] for i in range(1,rows+1)))  #dry matter percentage <=40  
  m.add_constr(mip.xsum(y[i,j]*e for i in range(1,rows+1)) <= max)  #max energy production in a trimester

#Objective function
m.objective = mip.maximize(mip.xsum((y[i,j]*es*e - z[i,j]*s[i-1] - x[i,j]*c[i-1]) for i in range(1,rows+1) for j in range(1,column+1) ))

solution=m.optimize()
print("x:")
for i in range (1,rows+1):
  for j in range(1,column+1):
    print([round(x[i,j].x,2)],end=' ')
  print('\n')  

print("y:")
for i in range (1,rows+1):
  for j in range(1,column+1):
    print([round(y[i,j].x,2)],end=' ')
  print('\n') 

print("z:")
for i in range (1,rows+1):
  for j in range(1,column+1):
    print([round(z[i,j].x,2)],end=' ')
  print('\n')   

print(m.objective_value)
print(solution)

x:
[700.0] [1500.0] [700.0] [0.0] [0.0] [700.0] [1500.0] [0.0] 

[1350.0] [0.0] [450.0] [0.0] [1350.0] [0.0] [450.0] [0.0] 

[0.0] [1500.0] [1500.0] [0.0] [0.0] [1500.0] [1500.0] [0.0] 

[820.0] [1560.0] [820.0] [0.0] [820.0] [1560.0] [820.0] [0.0] 

[0.0] [565.24] [1080.0] [0.0] [0.0] [680.0] [855.24] [0.0] 

y:
[700.0] [1000.0] [881.43] [318.57] [0.0] [700.0] [1125.0] [375.0] 

[1350.0] [0.0] [450.0] [0.0] [1350.0] [0.0] [450.0] [0.0] 

[0.0] [1500.0] [1393.81] [106.19] [0.0] [1500.0] [1375.0] [125.0] 

[820.0] [1560.0] [820.0] [0.0] [820.0] [1560.0] [820.0] [0.0] 

[0.0] [565.24] [1080.0] [0.0] [0.0] [680.0] [855.24] [0.0] 

z:
[0.0] [500.0] [318.57] [0.0] [0.0] [0.0] [375.0] [0.0] 

[0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [0.0] 

[0.0] [0.0] [106.19] [0.0] [0.0] [0.0] [125.0] [0.0] 

[0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [0.0] 

[0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [0.0] 

2861373.9254127136
OptimizationStatus.OPTIMAL
